In [6]:
from nltk.probability import FreqDist
import csv
import pandas as pd
from nltk.metrics import agreement

# Confusion Matrix

In [87]:
class ConfusionMatrix(object):


    def __init__(self, reference, test, sort_by_count=False):
        """
        Construct a new confusion matrix from a list of reference
        values and a corresponding list of test values.

        :type reference: list
        :param reference: An ordered list of reference values.
        :type test: list
        :param test: A list of values to compare against the
            corresponding reference values.
        :raise ValueError: If ``reference`` and ``length`` do not have
            the same length.
        """
        if len(reference) != len(test):
            raise ValueError("Lists must have the same length.")

        # Get a list of all values.
        if sort_by_count:
            ref_fdist = FreqDist(reference)
            test_fdist = FreqDist(test)

            def key(v):
                return -(ref_fdist[v] + test_fdist[v])

            values = sorted(set(reference + test), key=key)
        else:
            values = sorted(set(reference + test))

        # Construct a value->index dictionary
        indices = dict((val, i) for (i, val) in enumerate(values))

        # Make a confusion matrix table.
        confusion = [[0 for val in values] for val in values]
        max_conf = 0  # Maximum confusion
        for w, g in zip(reference, test):
            confusion[indices[w]][indices[g]] += 1
            max_conf = max(max_conf, confusion[indices[w]][indices[g]])

        #: A list of all values in ``reference`` or ``test``.
        self._values = values
        #: A dictionary mapping values in ``self._values`` to their indices.
        self._indices = indices
        #: The confusion matrix itself (as a list of lists of counts).
        self._confusion = confusion
        #: The greatest count in ``self._confusion`` (used for printing).
        self._max_conf = max_conf
        #: The total number of values in the confusion matrix.
        self._total = len(reference)
        #: The number of correct (on-diagonal) values in the matrix.
        self._correct = sum(confusion[i][i] for i in range(len(values)))

    def __getitem__(self, li_lj_tuple):
        """
        :return: The number of times that value ``li`` was expected and
        value ``lj`` was given.
        :rtype: int
        """
        (li, lj) = li_lj_tuple
        i = self._indices[li]
        j = self._indices[lj]
        return self._confusion[i][j]

    def __repr__(self):
        return "<ConfusionMatrix: %s/%s correct>" % (self._correct, self._total)

    def __str__(self):
        return self.pretty_format()

    def pretty_format(
        self,
        show_percents=False,
        values_in_chart=True,
        truncate=None,
        sort_by_count=False,
    ):
        """
        :return: A multi-line string representation of this confusion matrix.
        :type truncate: int
        :param truncate: If specified, then only show the specified
            number of values.  Any sorting (e.g., sort_by_count)
            will be performed before truncation.
        :param sort_by_count: If true, then sort by the count of each
            label in the reference data.  I.e., labels that occur more
            frequently in the reference label will be towards the left
            edge of the matrix, and labels that occur less frequently
            will be towards the right edge.

        @todo: add marginals?
        """
        confusion = self._confusion

        values = self._values
        if sort_by_count:
            values = sorted(
                values, key=lambda v: -sum(self._confusion[self._indices[v]])
            )

        if truncate:
            values = values[:truncate]

        if values_in_chart:
            value_strings = ["%s" % val for val in values]
        else:
            value_strings = [str(n + 1) for n in range(len(values))]

        # Construct a format string for row values
        valuelen = max(len(val) for val in value_strings)
        value_format = "%" + repr(valuelen) + "s | "
        # Construct a format string for matrix entries
        if show_percents:
            entrylen = 6
            entry_format = "%5.1f%%"
            zerostr = "     ."
        else:
            entrylen = len(repr(self._max_conf))
            entry_format = "%" + repr(entrylen) + "d"
            zerostr = " " * (entrylen - 1) + "."

        # Write the column values.
        s = ""
        for i in range(valuelen):
            s += (" " * valuelen) + " |"
            for val in value_strings:
                if i >= valuelen - len(val):
                    s += val[i - valuelen + len(val)].rjust(entrylen + 1)
                else:
                    s += " " * (entrylen + 1)
            s += " |\n"

        # Write a dividing line
        s += "%s-+-%s+\n" % ("-" * valuelen, "-" * ((entrylen + 1) * len(values)))

        # Write the entries.
        for val, li in zip(value_strings, values):
            i = self._indices[li]
            s += value_format % val
            for lj in values:
                j = self._indices[lj]
                if confusion[i][j] == 0:
                    s += zerostr
                elif show_percents:
                    s += entry_format % (100.0 * confusion[i][j] / self._total)
                else:
                    s += entry_format % confusion[i][j]
                if i == j:
                    prevspace = s.rfind(" ")
                    s = s[:prevspace] + "<" + s[prevspace + 1 :] + ">"
                else:
                    s += " "
            s += "|\n"

        # Write a dividing line
        s += "%s-+-%s+\n" % ("-" * valuelen, "-" * ((entrylen + 1) * len(values)))

        # Write a key
        s += "(row = " + name_1 + "; col = " + name_2 + ")" + "\n"
        if not values_in_chart:
            s += "Value key:\n"
            for i, value in enumerate(values):
                s += "%6d: %s\n" % (i + 1, value)

        return s


    def key(self):
        values = self._values
        str = "Value key:\n"
        indexlen = len(repr(len(values) - 1))
        key_format = "  %" + repr(indexlen) + "d: %s\n"
        for i in range(len(values)):
            str += key_format % (i, values[i])

        return str

## Confusion Matrix

In [88]:
df = pd.read_csv('files/annotator_agreement_all_final.csv', sep = ';')

In [89]:
df.head(5)

,Headlines,vlad_entity,vlad,shanet_entity,shanet,dhruv_entity,dhruv,spacy_entity,spacy
0,This Land Was Made For You And Me' — Or Was It?,land,LOC,Land,LOC,Land,LOC,This Land Was Made,WORK_OF_ART
1,A Perspective On George Zimmerman That Every P...,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON
2,A Strategy To Combat Crime That McGruff Wished...,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON
3,Could You Go 100 Days Without Food? These Men ...,100 Days,DATE,Men,PERSON,100 Days,DATE,100 Days,DATE
4,Here’s A Map of American Exceptionalism That’s...,American Exceptionalism,NORP,American,NORP,American,NORP,American,NORP


In [104]:
name_1 = "Vlad"
# name_1 = "Shanet"
# name_1 = "Dhruv"
# name_2 = "Shanet"
# name_2 = "Dhruv"
name_2 = "Spacy"

def demo():
    list1=[]
    list2=[]

    with open('files/annotator_agreement_all_final.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0

        for row in csv_reader:
            # VLAD
            #list1.append(row[2])
            # SHANET
            #list1.append(row[4])
            # DHRUV
            list1.append(row[6])
            # SPACY
            list2.append(row[8])
            line_count += 1
        print(f'Processed {line_count-1} lines.')
    a1=list1[1:]
    a2=list2[1:]

    print(name_1, len(list1[1:]))
    print(name_2, len(list2[1:]))
    print("Confusion matrix:")

    print(ConfusionMatrix(a1, a2).pretty_format(sort_by_count=True))

if __name__ == "__main__":
    demo()

Processed 51 lines.
Vlad 51
Spacy 51
Confusion matrix:
            | W                     |
            | O                     |
            | R                     |
            | K                     |
            | _               U     |
            | O   P           N     |
            | F   E         M K   E |
            | _ D R T N     O N   V |
            | A A S I O G O N O L E |
            | R T O M R P R E W O N |
            | T E N E P E G Y N C T |
------------+-----------------------+
WORK_OF_ART |<8>. 1 . . . 1 . . . . |
       DATE | 2<5>. . . . . . . . 1 |
     PERSON | . 1<6>. . . . . . . . |
       TIME | . . .<6>. . . . . . . |
       NORP | . . . .<3>. 2 . . . . |
        GPE | . . 1 . .<3>. . . . . |
        ORG | 1 . . . . 1<2>. . . . |
      MONEY | . . . . . . .<3>. . . |
    UNKNOWN | 1 1 . . . . . .<.>. 1 |
        LOC | 1 . . . . . . . .<.>. |
      EVENT | . . . . . . . . . .<.>|
------------+-----------------------+
(row = Vlad; col = Spacy)



## Kappa score for annotators

In [59]:
df.head()

,Headlines,vlad_entity,vlad,shanet_entity,shanet,dhruv_entity,dhruv,spacy_entity,spacy
0,This Land Was Made For You And Me' — Or Was It?,land,LOC,Land,LOC,Land,LOC,This Land Was Made,WORK_OF_ART
1,A Perspective On George Zimmerman That Every P...,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON,George Zimmerman,PERSON
2,A Strategy To Combat Crime That McGruff Wished...,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON,McGruff,PERSON
3,Could You Go 100 Days Without Food? These Men ...,100 Days,DATE,Men,PERSON,100 Days,DATE,100 Days,DATE
4,Here’s A Map of American Exceptionalism That’s...,American Exceptionalism,NORP,American,NORP,American,NORP,American,NORP


In [97]:
inputfile='files/annotator_agreement_all_final.csv'
print(inputfile)
merged_df=pd.read_csv(inputfile, delimiter=';', header = 0, index_col=0)
merged_df.columns=["vlad_entity", "vlad", "shanet_entity", "shanet", "dhruv_entity", "dhruv", "spacy_entity", "spacy"]

print(merged_df.shape)

labels_matched_df = merged_df.dropna()

# Reformat the data into the form AnnotationTask
#  expects.
data = []
for idx, row in labels_matched_df.iterrows():
    data.append(("a1", idx, row["vlad"]))
    data.append(("a2", idx, row["spacy"]))

atask = agreement.AnnotationTask(data=data)

print("Percentage agreement:", atask.avg_Ao())
print("Cohen's Kappa:", atask.kappa())
#print("Fleiss's Kappa:", atask.multi_kappa())
#print("Krippendorf's alpha:", atask.alpha())

# This function maps labels into a numeric space,
# . so we can rely on the ordering of labels.
def priority_distance(left_label, right_label):
    mapped_labels = {
        "Critical": 4,
        "High": 3,
        "Medium": 2,
        "Low": 1,
    }
    left_i = mapped_labels[left_label]
    right_i = mapped_labels[right_label]

    return abs(left_i - right_i)

files/annotator_agreement_all_final.csv
(51, 8)
Percentage agreement: 0.5641025641025641
Cohen's Kappa: 0.4479600333055787
